In [1]:
#import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpim

try:
    STATS = np.load("stats.npy", allow_pickle=True)
except FileNotFoundError as fnf:
    print("Not found stats file.")
    STATS = []
    
FRUITS = ['Grapefruit Pink', 'Grapefruit White', 'Limes','Lemon', 'Mandarine',
          'Avocado', 'Mango', 'Mango Red', 'Maracuja', 'Guava']

training_path = "fruits-360_dataset/Training"
test_path = "fruits-360_dataset/Test"

Not found stats file.


In [2]:
def load_data(directory_path, fruit_restriction=None):
    X = []
    y = []

    labels = []
    for i, label in enumerate(os.listdir(directory_path)):
            labels.append(label)

    for label in labels:
        if FRUITS and label not in FRUITS:
            continue
        print(f"Adding: {label}")
        class_num = labels.index(label)
        path = os.path.join(directory_path, label)
        for img in os.listdir(path):
            img_path = os.path.join(path, img)

            image = mpim.imread(img_path)

            X.append(image)
            y.append(class_num)


#         plt.figure()    
#         plt.imshow(image, cmap='gray')
#         plt.show()

    X = np.array(X)        
    y = np.array(y)
    
    return X, y      

In [3]:
X_training, y_training = load_data(training_path)
print("\nCreated training array")    
print(f"X shape: {X_training.shape}")
print(f"y shape: {y_training.shape}")

Adding: Limes
Adding: Grapefruit Pink
Adding: Mandarine
Adding: Mango
Adding: Mango Red
Adding: Lemon
Adding: Avocado
Adding: Guava
Adding: Grapefruit White
Adding: Maracuja

Created training array
X shape: (4777, 100, 100, 3)
y shape: (4777,)


In [4]:
X_test, y_test = load_data(test_path)
print("\nCreated test array")    
print(f"X shape: {X_test.shape}")
print(f"y shape: {y_test.shape}")

Adding: Limes
Adding: Grapefruit Pink
Adding: Mandarine
Adding: Mango
Adding: Mango Red
Adding: Lemon
Adding: Avocado
Adding: Guava
Adding: Grapefruit White
Adding: Maracuja

Created test array
X shape: (1609, 100, 100, 3)
y shape: (1609,)
